<a href="https://colab.research.google.com/github/2020-nlp-c/nlp-statisticsmodel/blob/master/jyp/spamfiltering_encoded.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

In [3]:
df=pd.DataFrame({"tokens":["me free lottery","free get free you","you free scholarship","free to contact me","you won award","you ticket lottery"],
    "spam":[1,1,0,0,0,1]})
df

,tokens,spam
0,me free lottery,1
1,free get free you,1
2,you free scholarship,0
3,free to contact me,0
4,you won award,0
5,you ticket lottery,1


In [11]:
class spamfiltering:
    def __init__(self,df):
        self.df=df

    def make_table(self):
        tokens_ls=[x.split() for x in self.df["tokens"]]
        temp_table=[]

        for i,tokens in enumerate(tokens_ls):
            for word in tokens:
                temp_table.append({"word":word,"tokens_cat":i})
        
        temp_table=pd.DataFrame(temp_table)
        pivot_table=temp_table.merge(df, how="left",left_on="tokens_cat",right_index=True).pivot_table(index="spam",columns="word",values="tokens",aggfunc="count").fillna(0)
        
        table=pivot_table.T

        return table

    def score(self,k,string):
        temp_table=self.make_table().copy()
        
        for word in string.split():
            if word not in self.make_table().index:
                temp_table.loc[word]=[0,0]
        
        total_table=temp_table.T
        total_table["total"]=total_table.sum(axis=1)
        
        table=total_table.T
        
        log_table=np.log((table+k)/(table.T["total"]+2*k))
        
        prior_normal=table.loc["total"][0]/table.loc["total"].sum()
        prior_spam=table.loc["total"][1]/table.loc["total"].sum()

        normal_log_ls=[log_table.loc[word][0] for word in string.split() ]
        normal_prob=np.exp(sum(normal_log_ls)+np.log(prior_normal))

        spam_log_ls=[log_table.loc[word][1] for word in string.split()]
        spam_prob=np.exp(sum(spam_log_ls)+np.log(prior_spam))

        return normal_prob/(normal_prob+spam_prob),spam_prob/(normal_prob+spam_prob)

In [12]:
spamfiltering(df).score(0.5,"free lottery")

(0.12500000000000008, 0.8749999999999999)

In [13]:
spamfiltering(df).score(0.5,"free dksldkjf")

(0.41666666666666685, 0.5833333333333331)

In [14]:
spamfiltering(df).score(0.5,"ghi asdlf we")

(0.5, 0.5)

In [15]:
spamfiltering(df).make_table()

spam,0,1
word,,
award,1.0,0.0
contact,1.0,0.0
free,2.0,3.0
get,0.0,1.0
lottery,0.0,2.0
me,1.0,1.0
scholarship,1.0,0.0
ticket,0.0,1.0
to,1.0,0.0


In [ ]:
spamfiltering(df)